In [61]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import statistics
#import missingno as msno

In [62]:
path_file = 'U98.xlsx'
sheets_excel = pd.read_excel('U98.xlsx',sheet_name=None)

In [63]:
for name in list(sheets_excel.keys()):
    sheets_excel[name].to_csv(name+'.csv', index=False)

In [64]:
sheets_excel.keys()

dict_keys(['U1399Data', 'R1399P1', 'R1399P2', 'U1399P3S01', 'U1399P3S02', 'U1399P3S03', 'U1399P3S04', 'U1399P3S05', 'U1399P3S06', 'U1399P3S07', 'U1399P3S08', 'U1399P3S09', 'U1399P3S10', 'U1399P3S11', 'U1399P3S12', 'U1399P3S13', 'U1399P3S14', 'U1399P4S01', 'U1399P4S02', 'U1399P4S03', 'U1399P4S04'])

### P4_S01

In [111]:
DF_P4_S1 = pd.read_csv('U1399P4S01.csv')

In [112]:
DF_P4_S1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12755 entries, 0 to 12754
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Address        12755 non-null  int64  
 1   member         12755 non-null  int64  
 2   employed_w     12755 non-null  int64  
 3   ISCO_w         12755 non-null  int64  
 4   ISIC_w         12755 non-null  int64  
 5   status_w       12755 non-null  int64  
 6   hours_w        11858 non-null  float64
 7   days_w         11858 non-null  float64
 8   income_w_m     12755 non-null  int64  
 9   income_w_y     12755 non-null  int64  
 10  wage_w_m       12745 non-null  float64
 11  wage_w_y       12751 non-null  float64
 12  perk_w_m       12745 non-null  float64
 13  perk_w_y       12751 non-null  float64
 14  netincome_w_m  12755 non-null  int64  
 15  netincome_w_y  12755 non-null  int64  
 16  Fasl           12755 non-null  int64  
 17  year           12755 non-null  int64  
 18  DYCOL0

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون DYcol00 کامل nan است.

In [113]:
DF_P4_S1.drop(['DYCOL00'], axis=1, inplace=True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
همبستگی ستون‌های income_w , wage_w, netincome_w مورد برسی قرار گرفت و چون همستگی ستون income در سال‌های دیگه کم بود فقط ستون wage_w حذف میشود.

In [114]:
DF_P4_S1[['income_w_m', 'wage_w_m', 'netincome_w_m']].corr()

,income_w_m,wage_w_m,netincome_w_m
income_w_m,1.000000,0.762181,0.780017
wage_w_m,0.762181,1.000000,0.978651
netincome_w_m,0.780017,0.978651,1.000000


In [115]:
DF_P4_S1[['income_w_y', 'wage_w_y', 'netincome_w_y']].corr()

,income_w_y,wage_w_y,netincome_w_y
income_w_y,1.000000,0.835469,0.846165
wage_w_y,0.835469,1.000000,0.988116
netincome_w_y,0.846165,0.988116,1.000000


In [116]:
DF_P4_S1.drop(['wage_w_m'], axis=1 , inplace= True)
DF_P4_S1.drop(['wage_w_y'], axis=1 , inplace= True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
با توجه به اطلاعات زیر ستون‌های employed_w, status_w,  perk_w_m دارای imblance هستند بنابراین حذف می‌کنیم.

In [117]:
for col in DF_P4_S1.columns:
 print(f'{col :>15} {(DF_P4_S1[col].value_counts()/len(DF_P4_S1)).max():f}')

        Address 0.000706
         member 0.655978
     employed_w 0.916268
         ISCO_w 0.152803
         ISIC_w 0.155782
       status_w 0.753195
        hours_w 0.497530
         days_w 0.503646
     income_w_m 0.090474
     income_w_y 0.041552
       perk_w_m 0.923089
       perk_w_y 0.563309
  netincome_w_m 0.091102
  netincome_w_y 0.041160
           Fasl 0.262328
           year 1.000000


In [118]:
DF_P4_S1.drop(['employed_w', 'status_w', 'perk_w_m'], inplace=True, axis=1)

In [119]:
DF_P4_S1.isna().sum()

Address            0
member             0
ISCO_w             0
ISIC_w             0
hours_w          897
days_w           897
income_w_m         0
income_w_y         0
perk_w_y           4
netincome_w_m      0
netincome_w_y      0
Fasl               0
year               0
dtype: int64

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های day_w , hours_w با گروه بندی روی ISCO_w, ISIC_w پر می‌کنیم و ما بقی را با میانه کل داده‌ها پر می‌کنیم.

In [120]:
temp = DF_P4_S1.groupby(['ISCO_w', 'ISIC_w' ])[['hours_w','days_w']].transform('median')
#temp = np.floor(temp).astype(pd.Float64Dtype())
DF_P4_S1.loc[DF_P4_S1.hours_w.isna(), 'hours_w'] = temp.loc[DF_P4_S1.hours_w.isna(), 'hours_w']
DF_P4_S1.loc[DF_P4_S1.days_w.isna(), 'days_w'] = temp.loc[DF_P4_S1.days_w.isna(), 'days_w']

In [121]:
DF_P4_S1.isna().sum()

Address            0
member             0
ISCO_w             0
ISIC_w             0
hours_w           99
days_w           100
income_w_m         0
income_w_y         0
perk_w_y           4
netincome_w_m      0
netincome_w_y      0
Fasl               0
year               0
dtype: int64

In [122]:
DF_P4_S1.hours_w.fillna(DF_P4_S1.hours_w.median(),inplace= True)
DF_P4_S1.days_w.fillna(DF_P4_S1.days_w.median(), inplace= True)

In [123]:
DF_P4_S1.isna().sum()

Address          0
member           0
ISCO_w           0
ISIC_w           0
hours_w          0
days_w           0
income_w_m       0
income_w_y       0
perk_w_y         4
netincome_w_m    0
netincome_w_y    0
Fasl             0
year             0
dtype: int64

In [124]:
DF_P4_S1.dropna(inplace=True)

In [125]:
path_file = 'U1398_P4_S01'
DF_P4_S1.to_csv(path_file+'.csv', index=False)

### P4_S02

In [126]:
DF_P4_S2 = pd.read_csv('U1399P4S02.csv')
DF_P4_S2.head()

,Address,member,employed_s,ISCO_s,ISIC_s,status_s,agriculture,hours_s,days_s,cost_employment,cost_raw,cost_machinery,cost_others,cost_tax,sale,income_s_y,Fasl,year,DYCOL00
0,10002002808,1.0,1.0,7213.0,45200.0,5.0,2.0,9.0,7.0,NaN,7.000000e+07,10000000.0,20000000.0,NaN,3.400000e+08,240000000.0,1,98,NaN
1,10002002815,1.0,1.0,1412.0,56100.0,4.0,2.0,7.0,7.0,233000000.0,2.400000e+09,20000000.0,100000000.0,50000000.0,3.263000e+09,460000000.0,1,98,NaN
2,10002002815,1.0,1.0,5224.0,47213.0,5.0,2.0,5.0,4.0,NaN,1.100000e+09,2000000.0,5000000.0,25000000.0,1.172000e+09,40000000.0,1,98,NaN
3,10006006814,1.0,1.0,5224.0,47212.0,5.0,2.0,8.0,7.0,NaN,1.000000e+09,3000000.0,145000000.0,15000000.0,1.583000e+09,420000000.0,1,98,NaN
4,10006006812,1.0,1.0,5224.0,47212.0,5.0,2.0,8.0,7.0,NaN,3.000000e+08,10000000.0,6000000.0,NaN,4.200000e+08,104000000.0,1,98,NaN


In [127]:
DF_P4_S2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7312 entries, 0 to 7311
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Address          7312 non-null   int64  
 1   member           7311 non-null   float64
 2   employed_s       7311 non-null   float64
 3   ISCO_s           7310 non-null   float64
 4   ISIC_s           7310 non-null   float64
 5   status_s         7311 non-null   float64
 6   agriculture      7311 non-null   float64
 7   hours_s          7146 non-null   float64
 8   days_s           7147 non-null   float64
 9   cost_employment  6333 non-null   float64
 10  cost_raw         7117 non-null   float64
 11  cost_machinery   6871 non-null   float64
 12  cost_others      7034 non-null   float64
 13  cost_tax         6400 non-null   float64
 14  sale             7268 non-null   float64
 15  income_s_y       7311 non-null   float64
 16  Fasl             7312 non-null   int64  
 17  year          

In [128]:
DF_P4_S2.drop(['DYCOL00'], axis=1 , inplace= True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های cost جمع می‌شوند و در ستون جدیدی به نام Total_cost ذخیره می‌شوند.

In [129]:
del_columns = ['cost_employment', 'cost_raw', 'cost_machinery', 'cost_others', 'cost_tax']
DF_P4_S2.insert(9,'Total_cost', DF_P4_S2.loc[:,'cost_employment':'cost_tax' ].sum(axis=1))
DF_P4_S2.drop(del_columns, axis=1, inplace=True)

In [130]:
DF_P4_S2.isna().sum()

Address          0
member           1
employed_s       1
ISCO_s           2
ISIC_s           2
status_s         1
agriculture      1
hours_s        166
days_s         165
Total_cost       0
sale            44
income_s_y       1
Fasl             0
year             0
dtype: int64

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های day_s , hours_s با گروه بندی روی ISCO_s, ISIC_s پر می‌کنیم و ما بقی را با میانه کل داده‌ها پر می‌کنیم.

In [131]:
temp = DF_P4_S2.groupby(['ISCO_s', 'ISIC_s' ])[['hours_s','days_s']].transform('median')
#temp = np.floor(temp).astype(pd.Int16Dtype())
DF_P4_S2.loc[DF_P4_S2.hours_s.isna(), 'hours_s'] = temp.loc[DF_P4_S2.hours_s.isna(), 'hours_s']
DF_P4_S2.loc[DF_P4_S2.days_s.isna(), 'days_s'] = temp.loc[DF_P4_S2.days_s.isna(), 'days_s']

DF_P4_S2.hours_s.fillna(DF_P4_S2.hours_s.median(),inplace= True)
DF_P4_S2.days_s.fillna(DF_P4_S2.days_s.median(), inplace= True)

In [132]:
DF_P4_S2.isna().sum()

Address         0
member          1
employed_s      1
ISCO_s          2
ISIC_s          2
status_s        1
agriculture     1
hours_s         0
days_s          0
Total_cost      0
sale           44
income_s_y      1
Fasl            0
year            0
dtype: int64

In [133]:
DF_P4_S2.dropna(subset = ['member','employed_s','ISCO_s','ISIC_s','status_s','agriculture'],inplace=True)

In [134]:
DF_P4_S2.isna().sum()

Address         0
member          0
employed_s      0
ISCO_s          0
ISIC_s          0
status_s        0
agriculture     0
hours_s         0
days_s          0
Total_cost      0
sale           43
income_s_y      0
Fasl            0
year            0
dtype: int64

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون sale
همان طور که در کد زیر نشان داده شده است تقریبن در 100 درصد موارد nan در ستون income_s_y مقدار0 دارد.

In [135]:
DF_P4_S2.loc[DF_P4_S2.sale.isna()].income_s_y.value_counts() / DF_P4_S2.sale.isna().sum() * 100

income_s_y
0.0    100.0
Name: count, dtype: float64

In [136]:
DF_P4_S2.loc[DF_P4_S2.sale.isna()].status_s.value_counts()

status_s
6.0    43
Name: count, dtype: int64

In [137]:
DF_P4_S2.fillna(0, inplace=True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون employed_s به دلیل imblance حذف می‌کنیم.

In [138]:
for col in DF_P4_S2.columns:
 print(f'{col :>15} {(DF_P4_S2[col].value_counts()/len(DF_P4_S2)).max():f}')

        Address 0.000821
         member 0.780027
     employed_s 0.970178
         ISCO_s 0.189877
         ISIC_s 0.131464
       status_s 0.835021
    agriculture 0.778386
        hours_s 0.270725
         days_s 0.455130
     Total_cost 0.169083
           sale 0.066347
     income_s_y 0.059781
           Fasl 0.269083
           year 1.000000


In [139]:
DF_P4_S2.drop(['employed_s'],axis=1, inplace=True)

In [140]:
DF_P4_S2.isna().sum()

Address        0
member         0
ISCO_s         0
ISIC_s         0
status_s       0
agriculture    0
hours_s        0
days_s         0
Total_cost     0
sale           0
income_s_y     0
Fasl           0
year           0
dtype: int64

In [141]:
path_file = 'U1398_P4_S02'
DF_P4_S2.to_csv(path_file+'.csv', index=False)

### P4_S03

In [142]:
DF_P4_S3 = pd.read_csv('U1399P4S03.csv')
DF_P4_S3.head()

,Address,member,income_pension,income_rent,income_interest,income_aid,income_resale,income_transfer,Fasl,year,DYCOL00
0,10004004223,1,0358000000,NaN,NaN,NaN,NaN,NaN,1,98,NaN
1,10004004223,2,0262000000,NaN,NaN,NaN,NaN,NaN,1,98,NaN
2,10002002810,1,NaN,0060000000,NaN,NaN,NaN,30000000.0,1,98,NaN
3,10002002820,1,0165000000,NaN,NaN,NaN,NaN,NaN,1,98,NaN
4,10002002823,1,0212400000,NaN,NaN,NaN,30000000.0,NaN,1,98,NaN


In [143]:
DF_P4_S3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19137 entries, 0 to 19136
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Address          19137 non-null  int64  
 1   member           19137 non-null  int64  
 2   income_pension   8917 non-null   object 
 3   income_rent      6065 non-null   object 
 4   income_interest  11900 non-null  float64
 5   income_aid       9125 non-null   float64
 6   income_resale    5200 non-null   float64
 7   income_transfer  8429 non-null   float64
 8   Fasl             19137 non-null  int64  
 9   year             19137 non-null  int64  
 10  DYCOL00          0 non-null      float64
dtypes: float64(5), int64(4), object(2)
memory usage: 1.6+ MB


In [144]:
DF_P4_S3.drop(['DYCOL00'], axis = 1, inplace=True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های income_pension , income_rent به float تغییر می‌دهیم.

In [145]:
DF_P4_S3.income_pension = DF_P4_S3.income_pension.str.strip()
DF_P4_S3.loc[DF_P4_S3.income_pension.isin(['']) , 'income_pension' ] = np.nan
DF_P4_S3.income_pension = DF_P4_S3.income_pension.astype(pd.Float64Dtype())

DF_P4_S3.income_rent = DF_P4_S3.income_rent.str.strip()
DF_P4_S3.loc[DF_P4_S3.income_rent.isin(['']) , 'income_rent' ] = np.nan
DF_P4_S3.income_rent = DF_P4_S3.income_rent.astype(pd.Float64Dtype())


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های income جمع می‌شوند و در ستون جدیدی به نام Total_income ذخیره می‌شوند.

In [146]:
DF_P4_S3.insert(2,'Total_income', DF_P4_S3.loc[:,'income_pension':'income_transfer' ].sum(axis=1))
del_columns = ['income_pension','income_rent', 'income_interest','income_aid','income_resale','income_transfer']
DF_P4_S3.drop(del_columns,axis=1, inplace=True)

In [147]:
DF_P4_S3.isna().sum()

Address         0
member          0
Total_income    0
Fasl            0
year            0
dtype: int64

In [148]:
path_file = 'U1398_P4_S03'
DF_P4_S3.to_csv(path_file+'.csv', index=False)

### P4_S04

In [149]:
DF_P4_S4 = pd.read_csv('U1399P4S04.csv')
DF_P4_S4.head()

,Address,member,subsidy_number,subsidy_month,subsidy,Fasl,year,DYCOL00
0,10004004223,1,2,12,10920000,1,98,NaN
1,10004004220,1,4,12,21840000,1,98,NaN
2,10002002810,1,1,12,5460000,1,98,NaN
3,10002002820,1,4,12,21840000,1,98,NaN
4,10002002823,1,3,12,16380000,1,98,NaN


In [150]:
DF_P4_S4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19833 entries, 0 to 19832
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Address         19833 non-null  int64  
 1   member          19833 non-null  int64  
 2   subsidy_number  19833 non-null  int64  
 3   subsidy_month   19833 non-null  int64  
 4   subsidy         19833 non-null  int64  
 5   Fasl            19833 non-null  int64  
 6   year            19833 non-null  int64  
 7   DYCOL00         0 non-null      float64
dtypes: float64(1), int64(7)
memory usage: 1.2 MB


In [151]:
DF_P4_S4.drop(['DYCOL00'], axis = 1, inplace=True)

In [152]:
DF_P4_S4.isna().sum()

Address           0
member            0
subsidy_number    0
subsidy_month     0
subsidy           0
Fasl              0
year              0
dtype: int64

In [153]:
path_file = 'U1398_P4_S04'
DF_P4_S4.to_csv(path_file+'.csv', index=False)